# CFT Example

<div class="alert alert-block alert-info">
This example illustrates how to process data collected when conducting the <i>Cold Face Test (CFT)</i> procedure, i.e., how to compute parameters quantifying the reaction to the CFT and how to create specialized CFT plots.
</div>

## Setup and Helper Functions

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, colors_all, register_fausans_font

import pandas as pd
import biopsykit as bp
from biopsykit.protocols import CFT

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

register_fausans_font()
palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="talk", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
output_path = Path("../results/plots")
output_path.mkdir(exist_ok=True, parents=True)

## Create CFT Object

The [CFT](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.protocols.cft.html#biopsykit.protocols.cft.CFT) is a protocol and thus part of the Protocol API of `BioPsyKit`.

In [ ]:
cft = CFT()
cft

## Load Data

In [ ]:
hr_dict = bp.example_data.get_mist_hr_example()

# Alternatively: Use your own data
# hr_dict = bp.io.ecg.load_hr_phase_dict("<path-to-heart-rate-data-dictionary>")
df_hr = hr_dict["MIST2"]

df_hr.head()

Note: See [plotting.hr_plot()](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.signals.ecg.plotting.html#biopsykit.signals.ecg.plotting.hr_plot) for further information!

## Plot CFT Data

In [ ]:
cft_params

In [ ]:
fig, ax = cft.cft_plot(
    data=df_hr, plot_onset=False, plot_mean=False, plot_peak_brady=False, plot_poly_fit=False, plot_baseline=False,
    background_alpha=1,
    background_base_color=["#E7F0F4", "#CEE0EA", "#9CC1D5"],
    phase_names=["Baseline", "Cold Face Intervention", "Recovery"],
)

cft_params = cft.compute_cft_parameter(df_hr, return_dict=True)
cft_times = cft._cft_plot_get_cft_times(df_hr, 60, 60)

cft_times = {key: (val - df_hr.index[0]).total_seconds() for key, val in cft_times.items()}

df_plot = df_hr.copy()
df_plot.index -= df_plot.index[0]

mean_hr = cft_params["mean_hr_bpm"]
cft_start = cft_times["cft_start"]
cft_end = cft_times["cft_end"]

# Baseline HR
ax.hlines(
    y=cft_params["baseline_hr"],
    xmin=cft_times["plot_start"],
    xmax=cft_times["cft_start"],
    ls="-",
    lw=3,
    color="#2F586E",
)

# Mean HR during CFT
ax.hlines(
    y=mean_hr,
    xmin=cft_start,
    xmax=cft_end,
    ls="-",
    lw=3,
    color="#2F586E",
)

ax.scatter(
    x=(cft_params["peak_brady"] - df_hr.index[0]).total_seconds(),
    y=df_hr.loc[cft_params["peak_brady"]],
    zorder=5,
    marker="X",
    color="#2F586E",
)

ax.set_ylim([65, 125])

fig.tight_layout(pad=0.1)

fig.savefig(output_path.joinpath("img_cft_plot.pdf"), transparent=True)

Note: See [CFT.cft_plot()](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.protocols.html#biopsykit.protocols.CFT.cft_plot) for further information!